### Structure
1. Understanding the problem
2. Exploratory Data Analysis (EDA) & visualization
3. Model training, tuning & evaluation
4. Upload

### 1. Understanding the problem

**Goal:** It is your job to predict if a passenger survived the sinking of the Titanic or not.
For each in the test set, you must predict a 0 or 1 value for the variable.

**Metric:**
Your score is the percentage of passengers you correctly predict. This is known as accuracy.

**Submission File Format:**
You should submit a csv file with exactly 418 entries plus a header row. Your submission will show an error if you have extra columns (beyond PassengerId and Survived) or rows.

The file should have exactly 2 columns:
- PassengerId (sorted in any order)
- Survived (contains your binary predictions: 1 for survived, 0 for deceased)

### 2. EDA

#### 2.0 Data loading & exploration

In [ ]:
# Load libraries & datasets
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split


In [ ]:
def concat_dfs(df_train, df_test):
    ''' Concatenate train and test dataframes'''
    df_all = pd.concat([df_train, df_test], axis=0, ignore_index=True)
    return df_all

def split_dfs(df_all, train_size=891):
    ''' Split concatenated dataframe back to train and test dataframes'''
    df_train = df_all.iloc[:train_size, :].copy()
    df_train['Survived'] = df_train['Survived'].astype(int)  # Ensure 'Survived' is int
    df_test = df_all.iloc[train_size:, :].copy()
    return df_train, df_test

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

df_all = concat_dfs(df_train, df_test)

In [ ]:
df_all.head()

* `PassengerId` is the unique id of the row and it doesn't have any effect on target
* `Survived` is the target variable we are trying to predict (0 or 1):
* **1 = Survived**
* **0 = Not Survived**
* `Pclass` (Passenger Class) is the socio-economic status of the passenger and it is a categorical ordinal feature which has **3** unique values (**1**, **2** or **3**):
- **1 = Upper Class**
- **2 = Middle Class**
- **3 = Lower Class**
- `Name`, `Sex` and `Age` are self-explanatory
- `SibSp` is the total number of the passengers' siblings and spouse
- `Parch` is the total number of the passengers' parents and children
- `Ticket` is the ticket number of the passenger
- `Fare` is the passenger fare
- `Cabin` is the cabin number of the passenger
- `Embarked` is port of embarkation and it is a categorical feature which has 3 unique values (C, Q or S):
- **C = Cherbourg**
- **Q = Queenstown**
- **S = Southampton**

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_all['Sex'].value_counts()

In [ ]:
# survival rate of women vs men
female_survival_rate = df_train.loc[df_train['Sex'] == 'female', 'Survived'].mean().round(4)
male_survival_rate = df_train.loc[df_train['Sex'] == 'male', 'Survived'].mean().round(4)
print(f"Female survival rate: {female_survival_rate}")
print(f"Male survival rate: {male_survival_rate}")

In [ ]:
# Plotting a stacked age distribtion histogram on condition of survived or not
plt.figure(figsize=(10, 6))
sns.histplot(data=df_train, x='Age', hue='Survived', multiple='layer', bins=40)
plt.title('Age Distribution by Survival Status')

### 2.1 Missing Values
As seen below, some columns are missing values. Our function below shows the missing amounts and percentages in each of df_train and df_test.
- Training set is missing data on `Age`, `Embarked`, and `Cabin`.
- Test set is missing data on `Age`, `Fare`, and `Cabin` (and natually the target variable `Survived`).

It's convenient to treat the training and test sets as a total set when dealing with missing values, otherwise filled data may overfit to the respective set samples.

The count of missing values in `Embarked` and `Fare` are less than one percent. But `Age` is missing around 20% and `Cabin` is missing around 80%.

Missing values in `Embarked`, `Fare` and `Age` can be inferred with descriptive statistical methods, but this will not work for `Cabin`.



In [ ]:
def display_missing(df):
    ''' Display missing values in dataframe'''
    total = df.isnull().sum()
    percent = (df.isnull().sum()/df.isnull().count()).round(4)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

for df in [df_train, df_test]:
    print(display_missing(df))

In [ ]:
df_all.isnull().sum().sort_values(ascending=False)

**2.1.1 Age**

We first find out the correlation coefficient between `Age` and other numerical features. 

In [ ]:
# Correlations between numerical features in dataframe df_all
df_all_corr = df_all.select_dtypes(include=[np.number]).corr().unstack().sort_values(ascending=False).reset_index()
df_all_corr.columns = ['Feature_1', 'Feature_2', 'Correlation']
df_all_corr[df_all_corr['Feature_1'] == 'Age']

In [ ]:
df_all.info()

In order to increase accuracy, the `Sex` feature is included as the second level of groupby after `Pclass` when filling the missing `Age` values.

In [ ]:
# Print median age by Pclass and Sex
for (key, value) in df_all.groupby(['Pclass', 'Sex'])['Age'].median().items():
    print(f"Median age for Pclass {key[0]}, {key[1]}: {value}")

# plot pclass, sex, and median age on a bar chart
df_age_pclass_sex = df_all.groupby(['Pclass', 'Sex'])['Age'].median().unstack()
df_age_pclass_sex.plot(kind='bar', figsize=(4, 3))

In [ ]:
# filling the median age for each Pclass and Sex
df_all['Age'] = df_all.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

**2.1.2 Embarked**

A categorical feature that is only missing 2 entries. A [Google search](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html) shows they embarked in **Southampton**.

In [ ]:
df_all[df_all['Embarked'].isnull()]

In [ ]:
# filling the missing Embarked values with 'S' (Southampton)
df_all['Embarked'] = df_all['Embarked'].fillna('S')

**2.1.3 Fare**

The one person missing fare information is a Male with no Family, a unique Ticket and no Cabin. We take this fare as the mean Fare of similar persons.

In [ ]:
# boolean logic to find out if which tickets are unique in df_all
unique_tickets = df_all['Ticket'].value_counts()
df_all['IsUniqueTicket'] = df_all['Ticket'].map(lambda x: unique_tickets[x] == 1)

# Now we filter for our desired rows
mask = ((df_all['Pclass'] == 3) & 
        (df_all['SibSp'] == 0) & 
        (df_all['Parch'] == 0) & 
        (df_all['Sex'] == 'male') & 
        (df_all['IsUniqueTicket']) &
        (df_all['Cabin'].isnull())
        )

df_all.loc[mask, 'Fare'].describe()

In [ ]:
mean_fare = df_all.loc[mask, 'Fare'].mean()
df_all.loc[df_all['Fare'].isnull(), 'Fare'] = mean_fare

**2.1.4 Cabin** 

We extract the Deck from each Cabin. Null-values are filled with 'Z' and the single 'T' value is replaced with 'Z'.

In [ ]:
df_all['Deck'] = df_all['Cabin'].str[0].fillna('Z')
# Change all 'T' to 'Z' since there is only one 'T'
df_all['Deck'] = df_all['Deck'].replace('T', 'Z')

In [ ]:
# plot survival rate by Deck
df_train, df_test = split_dfs(df_all)
deck_survival_rate = df_train.groupby('Deck')['Survived'].mean()
deck_survival_rate.plot(kind='bar', figsize=(6, 4))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylabel('Survival Rate')

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 10))

sns.heatmap(df_train.select_dtypes(include=np.number).drop(['PassengerId'], axis=1).corr(), ax=axs[0], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 10})
sns.heatmap(df_test.select_dtypes(include=np.number).drop(['PassengerId'], axis=1).drop(['Survived'], axis=1).corr(), ax=axs[1], annot=True, square=True, cmap='coolwarm', annot_kws={'size': 10})

for i in range(2):    
    axs[i].tick_params(axis='x', labelsize=10)
    axs[i].tick_params(axis='y', labelsize=10)
    axs[i].set_yticklabels(axs[i].get_yticklabels(), rotation=0)

axs[0].set_title('Training Set Correlations', size=10)
axs[1].set_title('Test Set Correlations', size=10)

plt.show()

### 2.2 Feature engineering

**2.2.1 Title**

We extract title from name and use as feature.

In [ ]:
df_all['Title'] = df_all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

In [ ]:
df_all['Title'].value_counts().plot(kind='bar')

In [ ]:
df_all['Title'] = df_all['Title'].replace({'Dona': 'Miss', 'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Lady': 'Nobility', 'Countess': 'Nobility', 'the Countess': 'Nobility', 'Don': 'Nobility', 'Capt': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Dr': 'Officer', 'Rev': 'Clergy', 'Sir': 'Nobility', 'Jonkheer': 'Nobility'})

In [ ]:
df_all['Title'].value_counts()

In [ ]:
# plot survival rate by title
df_train, df_test = split_dfs(df_all)

plt.figure(figsize=(10, 6))
sns.barplot(x='Title', y='Survived', data=df_train, errorbar=("ci",95))
plt.title('Survival Rate by Title')

**2.2.2 Married status**

Underlining the difference between married and unmarried women, although this may be captured in the `Title` feature.

In [ ]:
df_all['IsMarried'] = df_all['Title'].apply(lambda x: 1 if x in ['Mrs'] else 0)

In [ ]:
sns.barplot(x='Title', y='Survived', hue='IsMarried', data=df_all, alpha=0.7)

**2.2.3 FamilySize**

Calculating family size as Siblings and Spouse + Children and Parents + 1 for self

In [ ]:
df_all['FamilySize'] = df_all['SibSp'] + df_all['Parch'] + 1

**2.2.4 Fare pr. person**

In [ ]:
df_all['FarePrPerson'] = df_all['Fare'] / df_all['FamilySize']

**2.2.5 Surname**

In [ ]:
df_all['Surname'] = df_all['Name'].apply(lambda x: x.split(', ')[0])
df_all[df_all['FamilySize'] > 4].sort_values(by=['FamilySize','Surname','Title', 'Parch'], ascending=False)[
    ['Surname', 'Name', 'Title', 'Ticket', 'FamilySize', 'SibSp','Parch', 'Age', 'Fare', 'Deck', 'Survived',]][61:90]

### 3. Model training

In [ ]:
df_train, df_test = split_dfs(df_all)

In [ ]:
df_all.columns

In [ ]:
# random forest classifier with more features
from sklearn.ensemble import RandomForestClassifier

# Drop columns not needed
drop_cols = ['Name', 'Cabin']
df_all_drop = df_all.drop(columns=drop_cols)
df_train, df_test = split_dfs(df_all_drop)

# Only one-hot encode categorical columns
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns
X = pd.get_dummies(df_train, columns=categorical_cols)
X_test = pd.get_dummies(df_test, columns=categorical_cols)

# Align columns in train and test
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

y = df_train['Survived']

model = RandomForestClassifier(n_estimators=1000, max_depth=5, n_jobs=12, min_samples_leaf=3, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('models/rf_submission.csv', index=False)

In [ ]:
# plot model decision tree
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plot_tree(model.estimators_[0], feature_names=list(X.columns), filled=True)
plt.show()

In [ ]:
# xgboost classifier
import xgboost as xgb

y = df_train["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Fare"]

X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(df_test[features])

model = xgb.XGBClassifier(n_estimators=1000, max_depth=5, random_state=1)
model.fit(X, y)

predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': predictions})
output.to_csv('models/xgb_submission.csv', index=False)